In [1]:
import os
import dask.dataframe as dd
from sqlalchemy import create_engine

from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Obter variáveis de ambiente
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')
host = os.environ.get('DB_HOST')
port = os.environ.get('DB_PORT', '5432')
database = os.environ.get('DB_NAME')

# Criar string de conexão
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'

# Nome da tabela
table_name = 'ds_market'

# Carregar a tabela como um Dask DataFrame
# Nota: Dask requer uma coluna de índice para particionamento; ajuste conforme sua tabela
df_dask = dd.read_sql_table(
    table_name=table_name,
    con=connection_string,  # Passar a string de conexão diretamente
    index_col='date',  # Substitua 'id' por uma coluna indexável adequada
    npartitions=10
)

# Exemplo de EDA com Dask
print(df_dask.head())